In [69]:
from scipy.stats import norm, t
import scipy
import numpy as np
import pandas as pd
from datetime import datetime
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

# read those three csv files
prices = pd.read_csv("DailyPrices.csv")
date_index_prices = pd.read_csv("DailyPrices.csv", parse_dates=['Date']).set_index('Date')
all_other_facs = pd.read_csv("F-F_Research_Data_Factors_daily.csv", parse_dates=['Date']).set_index('Date')
mom_fac = pd.read_csv("F-F_Momentum_Factor_daily.csv", parse_dates=['Date']).set_index('Date')


In [70]:
# join factors data and add a column of constants to the data
all_facs = all_other_facs.join(mom_fac, how='right')

# define a list of stocks
stocks = ['AAPL', 'META', 'UNH', 'MA', 'MSFT' ,'NVDA', 'HD', 'PFE', 'AMZN' ,'BRK-B', 'PG', 'XOM', 'TSLA' ,'JPM' ,'V', 'DIS', 'GOOGL', 'JNJ', 'BAC', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom   ']


# calculate arithmetic E(r) in past 10 years
avg_factor_rets = data.loc['2013-1-31':'2023-1-31'].mean(axis=0)
avg_daily_rets = pd.Series()
for stock in stocks:
  # calculate betas
  model = sm.OLS(dataset[stock] - dataset['RF'], sm.add_constant(dataset[factors]))
  results = model.fit()
    
  # assume alpha = 0
  avg_daily_rets[stock] = (results.params[factors] * avg_factor_rets[factors]).sum() \
                          + avg_factor_rets['RF'] 


/tmp/ipykernel_714/170269260.py:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  avg_daily_rets = pd.Series()


MissingDataError: exog contains inf or nans

In [ ]:
geo_means = np.log(1 + avg_daily_rets) * 255  
geo_covariance = np.log(1 + all_returns[stocks]).cov() * 255
print(geo_means)
     

In [ ]:

# arithmetic annual returns: mean and covariance
arith_means = np.exp(geo_means + np.diagonal(geo_covariance.values) / 2) - 1

nstocks = geo_covariance.shape[0]
arith_covariance = np.empty((nstocks, nstocks), dtype=float)
for i in range(nstocks):
  for j in range(i, nstocks):
    mu_i, mu_j = geo_means.iloc[i], geo_means.iloc[j]
    sigma2_i, sigma2_j = geo_covariance.iloc[i, i], geo_covariance.iloc[j, j]
    sigma_ij = geo_covariance.iloc[i, j]
    arith_covariance[i, j] = np.exp(mu_i + mu_j + (sigma2_i + sigma2_j) / 2) * (np.exp(sigma_ij) - 1)
    arith_covariance[j, i] = arith_covariance[i, j]
arith_covariance = pd.DataFrame(arith_covariance, columns=stocks, index=stocks)

print(arith_means)

In [ ]:
def optimize_Sharpe(covar, expected_r, rf):
    # Define objective function
    def negative_Sharpe(w):
        returns = np.dot(expected_r, w)
        std = np.sqrt(w @ covar @ w.T)
        return -(returns - rf) / std

    # Define constraints
    constraints = [
        {"type": "eq", "fun": lambda w: np.sum(w) - 1}
    ]

    # Define bounds
    bounds = [(0, None)] * len(expected_r)

    # Define initial guess
    x0 = np.full(len(expected_r), 1/len(expected_r))

    # Use minimize function to solve optimization problem
    result = minimize(negative_Sharpe, x0, method="SLSQP", bounds=bounds, constraints=constraints)

    # Return the objective value (risk) and the portfolio weights
    return {"max_Sharpe_Ratio": -result.fun, "weights": result.x}
